# Dags Decoupling: Création de Dags simple combiné en un seul Dag 

![atelier3](https://user-images.githubusercontent.com/123757632/231910388-d2d83283-8a29-454c-9b3e-223787b8477b.png)

## Création Dags et définition des dépendances entre les DAG dans Airflow

Lorsque deux DAG ont des relations de dépendance, il convient d'envisager de les combiner en un seul DAG, qui est généralement plus simple à comprendre. Airflow offre également une meilleure représentation visuelle de dépendances pour les tâches sur le même DAG. Cependant, il n'est parfois pas pratique de mettre tous les éléments connexes tâches sur le même DAG.

## 1. Création des deux dags Parents 'dag en amont'

Pour la création des deux dags parents , il faut commencer par la création des deux fichirs 'parent_dag_1.py' et 'parent_dag_2.py' dans le dossier 'dags' .



In [ ]:
└───dags|── parent_dag_1.py                  
        └── parent_dag_2.py

### . parent_dag1.py

Le DAG **'dag_1'** sera planifié pour s'exécuter toutes les minutes à partir d'un moment donné dans le futur grâce à l'argument 
schedule_interval='*/1 * * * *' .

Le DAG comportera **deux tâches** : **"hello_python"** et **"goodbye_dummy"**.

* La tâche "hello_python" sera une **opération Python** 'PythonOperator' qui appellera une fonction "greeting" pour enregistrer un message de journalisation à travers **python_callable=greeting**.


```
def greeting():
    """Just check that the DAG is started in the log."""
    import logging
    logging.info('Bonjour le monde depuis le DAG 1')
```


* La tâche "au revoir" sera un **opérateur fictif** 'DummyOperator' qui indiquera la fin du DAG.

Enfin, la tâche "hello_python" sera liée **(via l'opérateur ">>")** à la tâche "goodbye_dummy", ce qui signifie que la tâche "goodbye_dummy"  ne sera exécutée qu'après la réussite de la tâche "hello_python".

### . parent_dag2.py


Le DAG **'dag_2'** sera planifié pour s'exécuter toutes les minutes à partir d'un moment donné dans le futur grâce à l'argument 
schedule_interval='*/1 * * * *' .

Le DAG comportera **deux tâches** : **"hello_python"** et **"goodbye_dummy"**.

* La tâche "hello_python" sera une **opération Python** 'PythonOperator' qui appellera une fonction "greeting" pour enregistrer un message de journalisation à travers **python_callable=greeting**.


```
def greeting():
    """Just check that the DAG is started in the log."""
    import logging
    logging.info('Bonjour le monde depuis le DAG 2')
```


* La tâche "au revoir" sera un **opérateur fictif** 'DummyOperator' qui indiquera la fin du DAG.

Enfin, la tâche "hello_python" sera liée **(via l'opérateur ">>")** à la tâche "goodbye_dummy", ce qui signifie que la tâche "goodbye_dummy"  ne sera exécutée qu'après la réussite de la tâche "hello_python".

### 2 . Création du dag enfant 'dag en aval'


In [ ]:
└───dags|── parent_dag_1.py                  
        └── parent_dag_2.py
        └── child_master_dag.py

### . child_master_dag.py

Le DAG sera planifié pour s'exécuter toutes les minutes à partir d'un moment donné dans le futur grâce à l'argument schedule_interval='*/1 * * * *'.

Le DAG comporte trois tâches : 
* **Deux capteurs externes "externalsensor1, externalsensor2" et un opérateur fictif "goodbye_dummy".** 
* Les capteurs externes sont utilisés pour surveiller l'état des DAGs **"dag_1"** et **"dag_2"**. Ils sont créés en utilisant **ExternalTaskSensor** et sont configurés pour attendre que les deux DAGs soient terminés **(external_task_id=None)**. Les capteurs vérifieront également l'existence des tâches cibles **(check_existence=True)** et se mettront en pause si l'attente dépasse 120 secondes **(timeout=120)**.

* L'opérateur fictif est un **DummyOperator** nommé **"goodbye_master"** et indique la fin du DAG.

Enfin, **les capteurs externes** sont liés **(via l'opérateur >>)** à l'opérateur fictif, ce qui signifie que l'opérateur fictif ne sera exécuté que lorsque les deux capteurs auront réussi à surveiller l'état des deux DAGs.
</br>

Le DAG "Dag_master" servira ainsi à déclencher les DAGs "dag_1" et "dag_2" et à effectuer des actions si les deux DAGs réussissent.